# Trader Decision Workflow (Now) — via `aipricepatterns.Client`

This notebook helps a trader make a decision **right now** based on analogs from `rlx-search` via the Python SDK.

Workflow:
1) Connect to `aipricepatterns.Client` (env: `AIPP_BASE_URL`, `AIPP_API_KEY`).
2) `client.search(...)` → matches + percentile forecast.
3) `client.get_pattern_metrics(...)` → probabilities/distribution/TP-SL hints.
4) Decision card: `LONG / SHORT / NO_TRADE`.
5) (Optional) Deep-dive: best match trajectory and quick backtest sanity-check.

Important: this is an engineering assistant. Not financial advice.

## 1) Environment Parameters and Configuration

Environment variables:
- `AIPP_BASE_URL` (e.g., `http://localhost:8787`)
- `AIPP_API_KEY` (if needed)

Decision parameters:
- `SYMBOL`, `INTERVAL`, `Q` (query length), `F` (forecast horizon), `TOP_K`
- `ANCHOR_TS_MS` (optional): pin the "last bar" to a specific historical time (ms epoch)
- Decision card thresholds: `MIN_UP_PROB_PCT`, `MIN_MEDIAN_PCT`

In [13]:
from __future__ import annotations

import os
from dataclasses import dataclass
from typing import Optional, Literal

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# --- SDK import (installed -> fallback to ../src) ---
try:
    from aipricepatterns import Client
except Exception:
    import sys
    from pathlib import Path

    sys.path.append(str((Path.cwd() / ".." / "src").resolve()))
    from aipricepatterns import Client

# --- Connection ---
# For production use: https://aipricepatterns.com/api/rust
# For local use: http://localhost:8787
BASE_URL = os.getenv("AIPP_BASE_URL", "https://aipricepatterns.com/api/rust")
API_KEY = os.getenv("AIPP_API_KEY")

client = Client(api_key=API_KEY, base_url=BASE_URL)
print("BASE_URL:", client.base_url)
print("API_KEY set:", bool(API_KEY))

# --- Parameters ---
SYMBOL = os.getenv("SYMBOL", "BTCUSDT")
INTERVAL = os.getenv("INTERVAL", "1h")
Q = int(os.getenv("Q", "40"))
F = int(os.getenv("F", "24"))
TOP_K = int(os.getenv("TOP_K", "10"))

# Optional: pin the last bar to a specific timestamp (ms since epoch)
ANCHOR_TS_MS: Optional[int]
_anchor = os.getenv("ANCHOR_TS_MS", "")
ANCHOR_TS_MS = int(_anchor) if _anchor.strip() else None

MIN_UP_PROB_PCT = float(os.getenv("MIN_UP_PROB_PCT", "60"))
MIN_MEDIAN_PCT = float(os.getenv("MIN_MEDIAN_PCT", "0.5"))

SYMBOL, INTERVAL, Q, F, TOP_K, ANCHOR_TS_MS, MIN_UP_PROB_PCT, MIN_MEDIAN_PCT

BASE_URL: https://aipricepatterns.com/api/rust
API_KEY set: False


('BTCUSDT', '1h', 40, 24, 10, None, 60.0, 0.5)

## 2) Analog Search + Percentile Forecast (Fan Chart)

- Top matches table (similarity/corr/rmse)
- Fan chart (p10/median/p90) in percent

In [14]:
search = client.search(
    symbol=SYMBOL,
    interval=INTERVAL,
    q=Q,
    f=F,
    top_k=TOP_K,
    anchor_ts=ANCHOR_TS_MS,
)

matches = search.get("matches", []) or []
forecast = search.get("forecast") or {}
meta = search.get("meta") or {}

print("meta:", {k: meta.get(k) for k in ["symbol", "interval", "queryLength", "forecastHorizon", "lastPrice", "queryWindowEndTs"] if k in meta})

# Matches table
if matches:
    dfm = pd.DataFrame(matches)
    keep = [c for c in ["id", "similarity", "deltaPct", "corr", "rmse", "start", "end"] if c in dfm.columns]
    display(dfm[keep].head(15))
else:
    dfm = pd.DataFrame()
    print("No matches returned (check symbol/interval, service, or filters).")

# Fan chart
median_raw = forecast.get("median")
p10_raw = forecast.get("p10")
p90_raw = forecast.get("p90")

if isinstance(median_raw, list) and isinstance(p10_raw, list) and isinstance(p90_raw, list) and len(median_raw) > 1:
    # Пересчитываем в проценты относительно первой точки
    base = median_raw[0]
    median = [(v - base) / base * 100 for v in median_raw]
    p10 = [(v - base) / base * 100 for v in p10_raw]
    p90 = [(v - base) / base * 100 for v in p90_raw]
    
    x = np.arange(len(median))
    fig = go.Figure()
    
    # p10-p90 area
    fig.add_trace(go.Scatter(
        x=np.concatenate([x, x[::-1]]),
        y=np.concatenate([p90, p10[::-1]]),
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line=dict(color='rgba(255,255,255,0)'),
        hoverinfo="skip",
        showlegend=True,
        name='p10–p90'
    ))
    
    # Median line
    fig.add_trace(go.Scatter(
        x=x,
        y=median,
        line=dict(color='rgb(0,100,80)', width=3),
        mode='lines',
        name='median'
    ))
    
    # Zero line
    fig.add_shape(
        type="line", line=dict(color="gray", width=1, dash="dash"),
        x0=0, x1=len(median)-1, y0=0, y1=0
    )
    
    # Устанавливаем диапазон оси Y от мин до макс
    all_vals = p10 + p90 + median
    fig.update_layout(
        title=f"Forecast Fan Chart (percent) — {SYMBOL} {INTERVAL}",
        xaxis_title="Bars ahead",
        yaxis_title="Return (%)",
        yaxis=dict(range=[min(all_vals), max(all_vals)]),
        template="plotly_white",
        height=400,
        margin=dict(l=20, r=20, t=40, b=20),
    )
    
    fig.show()
else:
    print("Forecast not available in response (missing forecast.median/p10/p90).")

meta: {'symbol': 'BTCUSDT', 'interval': '1h', 'queryLength': 40, 'forecastHorizon': 24, 'lastPrice': 88256.08, 'queryWindowEndTs': 1766232000000}


,id,similarity,corr,rmse
0,m-55383,0.9759,0.9518,0.3105
1,m-31603,0.9735,0.9470,0.3254
2,m-47921,0.9734,0.9467,0.3265
3,m-55382,0.9733,0.9466,0.3267
4,m-53367,0.9713,0.9426,0.3387
5,m-30774,0.9704,0.9409,0.3439
6,m-52024,0.9694,0.9388,0.3498
7,m-49618,0.9694,0.9387,0.3501
8,m-47920,0.9693,0.9386,0.3506
9,m-31602,0.9693,0.9386,0.3503


## 3) Pattern Metrics: Probabilities, Distribution, TP/SL Hints

`get_pattern_metrics` provides aggregates for the current window:
- `upProbPct`, `medianPct`, `medianCorr`, `medianRmse`, drawdown/time-to-peak
- `distribution.sigmaLevels` are often useful as TP/SL levels (in %)
- (Optional) `get_grid_stats` — additional "grid" volatility hint

In [15]:
metrics_resp = client.get_pattern_metrics(
    symbol=SYMBOL,
    interval=INTERVAL,
    q=Q,
    f=F,
)

metrics = metrics_resp.get("metrics") or {}
dist = metrics_resp.get("distribution") or {}

# Show key metrics (defensive: not all keys exist in every build)
key_order = [
    "medianPct",
    "upProbPct",
    "medianCorr",
    "medianRmse",
    "medianDrawdownPct",
    "medianTimeToPeakBars",
    "medianTimeToTroughBars",
]
view = {k: metrics.get(k) for k in key_order if k in metrics}
display(pd.DataFrame([view]))

# Extract TP/SL suggestion from sigma levels if present
sigma_levels = dist.get("sigmaLevels") or []

suggested_tp = None
suggested_sl = None
if isinstance(sigma_levels, list) and sigma_levels:
    # prefer 1σ if it exists
    one = None
    for s in sigma_levels:
        if str(s.get("label", "")).strip() == "1σ":
            one = s
            break
    if one is None:
        one = sigma_levels[0]

    suggested_tp = one.get("positive")
    suggested_sl = one.get("negative")

print("Suggested TP/SL (pct):", {"tp": suggested_tp, "sl": suggested_sl})

# Optional: grid stats (may not be available on all deployments)
try:
    grid = client.get_grid_stats(symbol=SYMBOL, interval=INTERVAL)
    # keep it compact
    keys = ["sigmaLevels", "terminalPercentiles", "volatility", "symbol", "interval"]
    grid_view = {k: grid.get(k) for k in keys if k in grid}
    print("grid_stats:", grid_view)
except Exception as e:
    print("grid_stats unavailable:", str(e))

,medianPct,upProbPct,medianCorr,medianRmse,medianDrawdownPct,medianTimeToPeakBars,medianTimeToTroughBars
0,-0.3124,46.875,0.9351,0.3604,1.5058,9.5,16.0


Suggested TP/SL (pct): {'tp': 3.943070334098983, 'sl': -3.319374355985724}
grid_stats: {'sigmaLevels': [{'k': 1.0, 'label': '1σ', 'negative': -3.1573, 'positive': 4.272, 'slBeforeTpProb': 0.06, 'slHitProb': 0.06, 'terminalAbove': 0.06, 'terminalBelow': 0.06, 'tpBeforeSlProb': 0.14, 'tpHitProb': 0.14}, {'k': 2.0, 'label': '2σ', 'negative': -6.8719, 'positive': 7.9866, 'slBeforeTpProb': 0.04, 'slHitProb': 0.04, 'terminalAbove': 0.02, 'terminalBelow': 0.04, 'tpBeforeSlProb': 0.02, 'tpHitProb': 0.02}, {'k': 3.0, 'label': '3σ', 'negative': -10.5865, 'positive': 11.7012, 'slBeforeTpProb': 0.04, 'slHitProb': 0.04, 'terminalAbove': 0.02, 'terminalBelow': 0.04, 'tpBeforeSlProb': 0.02, 'tpHitProb': 0.02}], 'terminalPercentiles': {'p01': -12.897, 'p05': -4.3167, 'p10': -2.2269, 'p25': -0.7774, 'p50': 0.8714, 'p75': 2.1023, 'p90': 3.5825, 'p95': 4.1133, 'p99': 5.2968}}


## 4) Decision Card: LONG / SHORT / NO_TRADE

Rule (simple and transparent):
- LONG: `upProbPct >= MIN_UP_PROB_PCT` and `medianPct >= MIN_MEDIAN_PCT`
- SHORT: `upProbPct <= 100 - MIN_UP_PROB_PCT` and `medianPct <= -MIN_MEDIAN_PCT`
- Otherwise: NO_TRADE

The decision card displays:
- bias
- horizon `F`
- `medianPct`, `upProbPct`
- suggested TP/SL (if available)
- disclaimer

In [16]:
up_prob = float(metrics.get("upProbPct", float("nan")))
median_pct = float(metrics.get("medianPct", float("nan")))

bias: Literal["LONG", "SHORT", "NO_TRADE"] = "NO_TRADE"

if np.isfinite(up_prob) and np.isfinite(median_pct):
    if up_prob >= MIN_UP_PROB_PCT and median_pct >= MIN_MEDIAN_PCT:
        bias = "LONG"
    elif up_prob <= (100.0 - MIN_UP_PROB_PCT) and median_pct <= -MIN_MEDIAN_PCT:
        bias = "SHORT"
    else:
        bias = "NO_TRADE"

card = {
    "symbol": SYMBOL,
    "interval": INTERVAL,
    "q": Q,
    "f": F,
    "bias": bias,
    "medianPct": median_pct if np.isfinite(median_pct) else None,
    "upProbPct": up_prob if np.isfinite(up_prob) else None,
    "suggested_tp_pct": suggested_tp,
    "suggested_sl_pct": suggested_sl,
    "anchor_ts_ms": ANCHOR_TS_MS,
}

print("\n=== DECISION CARD ===")
for k, v in card.items():
    print(f"{k:>16}: {v}")
print("\nDisclaimer: educational workflow, not financial advice.")


=== DECISION CARD ===
          symbol: BTCUSDT
        interval: 1h
               q: 40
               f: 24
            bias: NO_TRADE
       medianPct: -0.3124
       upProbPct: 46.875
suggested_tp_pct: 3.943070334098983
suggested_sl_pct: -3.319374355985724
    anchor_ts_ms: None

Disclaimer: educational workflow, not financial advice.


## 5) (Optional) Deep-dive: Best Match Trajectory

If `get_match_details` returns `series`, you can see "how the best analog developed" from the detection point and compare it with the median forecast.

In [17]:
if matches:
    top_id = matches[0].get("id")
else:
    top_id = None

if not top_id:
    print("No match id available.")
else:
    try:
        details = client.get_match_details(top_id)
        series = details.get("series")

        if not isinstance(series, list) or len(series) < (Q + 2):
            print("No usable 'series' in match details.")
        else:
            # series typically contains [query_window ... forecast_window]
            start_idx = max(0, Q - 1)
            future = series[start_idx:]
            base = future[0]
            path_pct = [(x - base) / base * 100 for x in future]
            x = np.arange(len(path_pct))

            fig = go.Figure()
            
            # Match path
            fig.add_trace(go.Scatter(
                x=x,
                y=path_pct,
                mode='lines',
                name=f"match {top_id}",
                line=dict(width=2)
            ))

            # Forecast median if available
            if isinstance(forecast.get("median"), list) and len(forecast["median"]) == len(path_pct):
                fig.add_trace(go.Scatter(
                    x=x,
                    y=forecast["median"],
                    mode='lines',
                    name="forecast median",
                    line=dict(width=3, dash='dash')
                ))

            # Zero line
            fig.add_shape(
                type="line", line=dict(color="gray", width=1, dash="dot"),
                x0=0, x1=len(path_pct)-1, y0=0, y1=0
            )

            fig.update_layout(
                title="Top Match Continuation (percent)",
                xaxis_title="Bars from detection",
                yaxis_title="Return (%)",
                template="plotly_white",
                height=400,
                margin=dict(l=20, r=20, t=40, b=20),
            )
            
            fig.show()
    except Exception as e:
        print("get_match_details failed:", str(e))

## 6) (Optional) Quick Backtest Sanity-check

This is **not** a full strategy validation, but a quick way to ensure that the chosen `q/f/min_prob` yield meaningful trades.

If `rlx-search` is running and the endpoint is accessible, you can perform a walk-forward backtest via the API.

In [18]:
# Tune these separately from decision thresholds
MIN_PROB = float(os.getenv("MIN_PROB", "0.6"))
STEP = int(os.getenv("STEP", str(max(6, F // 2))))

try:
    bt = client.backtest(
        symbol=SYMBOL,
        interval=INTERVAL,
        q=Q,
        f=F,
        step=STEP,
        top_k=min(TOP_K, 10),
        min_prob=MIN_PROB,
        include_stats=True,
        fee_pct=float(os.getenv("AIPP_FEE_PCT", "0.04")),
        slippage_pct=float(os.getenv("AIPP_SLIPPAGE_PCT", "0.00")),
    )

    stats = bt.get("stats") or {}
    print("backtest.stats:", {k: stats.get(k) for k in ["totalReturnPct", "sharpeRatio", "maxDrawdownPct", "winRate", "profitFactor"] if k in stats})

    # 1) Trade History Table
    trades = bt.get("results") or []
    if trades:
        dft = pd.DataFrame(trades)
        # Clean up columns for display
        cols = [c for c in ["entryTs", "exitTs", "side", "pnlPct", "prob", "entryPrice", "exitPrice"] if c in dft.columns]
        print("\nTrade History (Top 10):")
        display(dft[cols].head(10))
    else:
        print("No trades executed in backtest.")

    # 2) Equity Curve
    equity = bt.get("equity") or stats.get("equityCurve")
    if isinstance(equity, list) and equity:
        # Handle both list of floats and list of {ts, value}
        if isinstance(equity[0], dict):
            y_vals = [e.get("value") for e in equity]
            x_vals = [pd.to_datetime(e.get("ts"), unit='ms') for e in equity]
        else:
            y_vals = equity
            x_vals = np.arange(len(equity))

        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=x_vals,
            y=y_vals,
            mode='lines',
            name='Equity',
            line=dict(color='royalblue', width=2)
        ))
        
        # Tight Y-axis scaling
        y_min, y_max = min(y_vals), max(y_vals)
        y_padding = (y_max - y_min) * 0.1 if y_max > y_min else 0.1
        
        fig.update_layout(
            title=f"Backtest Equity Curve — {SYMBOL} {INTERVAL}",
            xaxis_title="Time / Trade #",
            yaxis_title="Equity",
            yaxis=dict(range=[y_min - y_padding, y_max + y_padding]),
            template="plotly_white",
            height=400,
            margin=dict(l=20, r=20, t=40, b=20),
        )
        fig.show()

    # 3) Trade Returns Distribution
    if trades:
        pnl_vals = [t.get("pnlPct", 0) for t in trades]
        fig_dist = px.histogram(
            pnl_vals, 
            nbins=30, 
            title="Trade Returns Distribution (%)",
            labels={'value': 'Return (%)'},
            template="plotly_white",
            color_discrete_sequence=['#00CC96']
        )
        fig_dist.update_layout(
            height=300, 
            showlegend=False,
            xaxis_title="Return (%)",
            yaxis_title="Count"
        )
        fig_dist.show()

except Exception as e:
    print("backtest unavailable:", str(e))

backtest.stats: {'totalReturnPct': 226.6413581346551, 'sharpeRatio': 0.6532932074463098, 'maxDrawdownPct': 89.06977298308053, 'winRate': 52.38238841978286, 'profitFactor': 1.0830062760477372}

Trade History (Top 10):


,exitTs,entryPrice,exitPrice
0,1586232000000,6843.82,7252.84
1,1586275200000,7143.00,7368.63
2,1586318400000,7322.07,7355.00
3,1586361600000,7318.04,7322.68
4,1586404800000,7353.41,7288.51
5,1586448000000,7276.83,7280.75
6,1586491200000,7286.34,7140.06
7,1586534400000,7282.68,6857.54
8,1586577600000,7175.78,6907.14
9,1586620800000,6766.43,6811.64
